# Linear regression in MLX

Markus Enzweiler, markus.enzweiler@hs-esslingen.de

This is a demo used in a Computer Vision & Machine Learning lecture. Feel free to use and contribute.

**Note: This requires a machine with an Apple SoC, e.g. M1/M2/M3 etc.**

See: https://github.com/ml-explore/mlx


## Setup

Adapt `packagePath` to point to the directory containing this notebeook.

In [ ]:
# Notebook id
nb_id = "linear_regression/mlx"

# Imports
import sys
import os

In [ ]:
# Package Path (folder of this notebook)

#####################
# Local environment #
#####################

package_path = "./"


#########
# Colab #
#########


def check_for_colab():
  try:
      import google.colab
      return True
  except ImportError:
      return False

# running on Colab?
on_colab = check_for_colab()

if on_colab:

    # assume this notebook is run from Google Drive and the whole
    # cv-ml-lecture-notebooks repo has been setup via setupOnColab.ipynb

    # Google Drive mount point
    gdrive_mnt = '/content/drive'

    ##########################################################################
    # Ensure that this is the same as gdrive_repo_root in setupOnColab.ipynb #
    ##########################################################################
    # Path on Google Drive to the cv-ml-lecture-notebooks repo
    gdrive_repo_root = f"{gdrive_mnt}/MyDrive/cv-ml-lecture-notebooks"

    # mount drive
    from google.colab import drive
    drive.mount(gdrive_mnt, force_remount=True)

    # set package path
    package_path = f"{gdrive_repo_root}/{nb_id}"

# check whether package path exists
if not os.path.isdir(package_path):
  raise FileNotFoundError(f"Package path does not exist: {package_path}")

print(f"Package path: {package_path}")


In [ ]:
# Additional imports

# Repository Root
repo_root = os.path.abspath(os.path.join(package_path, "..", ".."))
# Add the repository root to the system path
if repo_root not in sys.path:
    sys.path.append(repo_root)

# Package Imports
from nbutils import requirements as nb_reqs
from nbutils import colab as nb_colab
from nbutils import git as nb_git
from nbutils import exec as nb_exec
from nbutils import data as nb_data

In [ ]:
# Install requirements in the current Jupyter kernel
req_file = os.path.join(package_path, "requirements.txt")
nb_reqs.pip_install_reqs(req_file, on_colab)

In [ ]:
# Now we should be able to import the additional packages
import mlx 
import mlx.core as mx

import numpy as np
import matplotlib.pyplot as plt

# Set the random seed for reproducibility
mx.random.seed(42)

## MLX

MLX provides composable function transformations, supporting automatic differentiation, automatic vectorization, and optimization of computation graphs. Computation graphs within MLX are dynamically constructed. A key feature of MLX is the use of (and optimization for) unified memory present in the Apple SoCs. 

See:
- https://github.com/ml-explore/mlx
- https://ml-explore.github.io/mlx/build/html/quick_start.html

## Linear regression

### Create some data based on adding noise to a known linear function

In [ ]:
# Creating a function f(x) with a slope of 2 and bias of 1, e.g. f(x) = 2x + 1
# and added Gaussian noise

# True parameters
w_true = 2
b_true = 1
params_true = mx.array([w_true, b_true])

X = mx.arange(-5, 5, 0.1)
Y = w_true*X + b_true + 2 * mx.random.normal(X.shape)

# Visualize
plt.scatter(X, Y, alpha=0.5)
plt.title("Scatter plot of f(x) = 2x + 1 + Gaussian noise")
plt.show()

### Linear model and loss

Our linear regression model is $ y = f(x) = w \cdot x + b$. We solve for $w$ and $b$ using gradient descent. 

In [ ]:
def lin_model(params, x):
    w,b = params
    return w*x + b

We uses mean squared error loss between the predictions of our model and true values. 

In [ ]:
def loss_fn(params, x, y):
    y_pred = lin_model(params, x)
    return mx.mean(mx.square(y - y_pred))

### Optimization via gradient descent

Initialize parameters $w$ and $b$ randomly.

In [ ]:
params = 1e-2 * mx.random.normal((2,))
w,b = params
print(f"Initial weight : {w}")
print(f"Initial bias   : {b}")

Optimize via gradient descent

In [ ]:
# Hyperparameters
num_iters = 10000
learning_rate = 3e-4

# Function that computes loss and its gradient
loss_and_grad_fn = mx.value_and_grad(loss_fn)

# Loop over the number of iterations
for it in range(num_iters):

    # gradient of loss function (vectorized)
    loss, loss_grad = loss_and_grad_fn(params, X, Y)

    # update parameters via gradient descent update rules
    params -= learning_rate * loss_grad

    # Evaluate the parameters explicitly, because MLX uses lazy evaluation
    mx.eval(params)

    # Give some status output once in a while
    if it % 500 == 0 or it == num_iters - 1:
        w,b = params  
        error_norm = mx.sum(mx.square(params - params_true)) ** 0.5
        print(f"Iteration {it:5d} | Loss {loss.item():>10.5f} | " 
              f"w {w.item():> 8.5f} | b {b.item():> 8.5f} | Error norm {error_norm.item():>.5f}")


    
w,b = params    
print(f"Final weight after optimization : {w.item():.5f} (true: {w_true})")
print(f"Final bias after optimization   : {b.item():.5f} (true: {b_true})")       

Visualize linear fit

In [ ]:
# Visualize
plt.scatter(X, Y, alpha=0.5)
plt.title("Scatter plot of f(x) = 2x + 1 + Gaussian noise")

# Plot the recovered line
Y_model = lin_model(params, X)
plt.plot(X.tolist(), Y_model.tolist(), color='red')

plt.legend(["data", f"f(x) = {w.item():.3f}x + {b.item():.3f}"])
plt.show()